In [1]:
import os
os.environ['TF_KERAS'] = '1'

In [2]:
import numpy as np
from bert4keras.backend import keras
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.snippets import to_array

2022-04-11 16:31:01.194624: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
config_path = './user_data/NEZHA-Base/bert_config.json'
checkpoint_path = './user_data/NEZHA-Base/model.ckpt-900000'
dict_path = './user_data/NEZHA-Base/vocab.txt'

In [5]:
tokenizer = Tokenizer(dict_path, do_lower_case=True)  # 建立分词器
model = build_transformer_model(config_path, checkpoint_path, model='nezha')  # 建立模型，加载权重

In [9]:
token_ids, segment_ids = tokenizer.encode('system')
token_ids, segment_ids = to_array([token_ids], [segment_ids])

In [11]:
model.predict([token_ids, segment_ids]).shape

(1, 3, 768)

In [2]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter

In [3]:
log_df = pd.read_csv('./src/new_log.csv')

In [6]:
log_df = log_df.sort_values(by='msg_lower')

In [7]:
from drain3 import TemplateMiner #开源在线日志解析框架
from drain3.file_persistence import FilePersistence
from drain3.template_miner_config import TemplateMinerConfig

In [8]:
config = TemplateMinerConfig()
config.load('./drain3.ini') ## 这个文件在drain3的github仓库里有
config.profiling_enabled = False

In [9]:
drain_file = 'comp_a_sellog'
persistence = FilePersistence(drain_file + '.bin')
template_miner = TemplateMiner(persistence, config=config)

In [12]:
for msg in log_df.msg_lower.tolist():
    template_miner.add_log_message(msg)
temp_count = len(template_miner.drain.clusters)

In [13]:
template_dic = {}

for cluster in template_miner.drain.clusters:
    template_dic[cluster.cluster_id] = cluster.size

In [14]:
temp_count_f = len(template_dic)

In [15]:
temp_count_f

187

In [16]:
def match_template(df, template_miner, template_dic):
    msg = df.msg_lower
    cluster = template_miner.match(msg) # 匹配模板，由开源工具提供
    if cluster and cluster.cluster_id in template_dic:
        df['template_id'] = cluster.cluster_id # 模板id
        df['template'] = cluster.get_template() # 具体模板
    else:
        df['template_id'] = 'None' # 没有匹配到模板的数据也会记录下来，之后也会用作一种特征。
        df['template'] = 'None'
    return df

In [17]:
log_df = log_df.apply(match_template, template_miner=template_miner, template_dic=template_dic, axis=1)

In [18]:
log_df.to_csv('./src/log_template.csv', index=False)

In [19]:
log_df

,sn,time,msg,server_model,msg_lower,msg_id,template_id,template
266,SERVER_12,2020-06-11 10:23:43,Add-in Card #0x1b | Device Present,SM3,add-in card #0x1b | device present,8,1,add-in card #<:HEX:> | device present
874,SERVER_481,2020-08-18 11:02:55,Add-in Card #0x1b | Device Present | Asserted,SM4,add-in card #0x1b | device present | asserted,29,2,add-in card #<:HEX:> | device present | asserted
461193,SERVER_25703,2020-10-20 11:10:01,Add-in Card #0x1b | Device Present | Asserted,SM4,add-in card #0x1b | device present | asserted,29,2,add-in card #<:HEX:> | device present | asserted
461190,SERVER_35,2020-08-20 08:32:13,Add-in Card #0x1b | Device Present | Asserted,SM4,add-in card #0x1b | device present | asserted,29,2,add-in card #<:HEX:> | device present | asserted
856,SERVER_25708,2020-07-25 12:43:19,Add-in Card #0x1b | Device Present | Asserted,SM4,add-in card #0x1b | device present | asserted,29,2,add-in card #<:HEX:> | device present | asserted
...,...,...,...,...,...,...,...,...
92403,SERVER_13653,2020-06-05 11:45:49,Watchdog2 IPMI_Watchdog | Timer expired | Ass...,SM13,watchdog2 ipmi_watchdog | timer expired | asse...,296,187,watchdog2 ipmi watchdog | <:*:> <:*:> | <:*:>
484491,2deae48ca05d,2020-07-07 08:36:04,Watchdog2 IPMI_Watchdog | Timer expired | Ass...,SM13,watchdog2 ipmi_watchdog | timer expired | asse...,296,187,watchdog2 ipmi watchdog | <:*:> <:*:> | <:*:>
338244,SERVER_26643,2020-04-19 01:04:19,Watchdog2 IPMI_Watchdog | Timer interrupt | A...,SM59,watchdog2 ipmi_watchdog | timer interrupt | as...,708,187,watchdog2 ipmi watchdog | <:*:> <:*:> | <:*:>
338179,SERVER_12218,2020-05-12 12:43:50,Watchdog2 IPMI_Watchdog | Timer interrupt | A...,SM59,watchdog2 ipmi_watchdog | timer interrupt | as...,708,187,watchdog2 ipmi watchdog | <:*:> <:*:> | <:*:>


In [49]:
log_df[log_df['template'].apply(lambda x: len(x.split(' '))) == 1]

,sn,time,msg,server_model,new_msg,template_id,template
369063,SERVER_19566,2020-07-06 11:54:40,Reserved |,SM39,reserved |,165,reserved


In [19]:
label1 = pd.read_csv('./data/preliminary_train_label_dataset.csv')
label2 = pd.read_csv('./data/preliminary_train_label_dataset_s.csv')
label_df = pd.concat([label1, label2]).reset_index(drop=True)

In [20]:
label_df = label_df.drop_duplicates().reset_index(drop=True)

In [21]:
label_df['fault_time'] = pd.to_datetime(label_df['fault_time'])
log_df['time'] = pd.to_datetime(log_df['time'])

In [22]:
label_df['start_fault_time'] = label_df['fault_time'] - pd.Timedelta(pd.offsets.Hour(3))
data = []
for i in tqdm(range(label_df.shape[0])):
    row = label_df.iloc[i]
    sn = row['sn']
    start_fault_time = row['start_fault_time']
    fault_time = row['fault_time']
    tmp = log_df[log_df['sn'] == sn]
    tmp = tmp[(tmp['time'] >= start_fault_time) & (tmp['time'] <= fault_time)].sort_values(by='time', ascending=False)
    data.append(np.unique(tmp['template_id'].values.tolist()))

  9%|██████████▏                                                                                                  | 1554/16604 [00:47<07:41, 32.60it/s]


KeyboardInterrupt: 

In [17]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter

In [18]:
log_df.shape, test_log_df.shape[0] + train_log_df.shape[0]

((493527, 7), 493527)

In [19]:
token2count = Counter()
token2count.update(log_df.iloc[:train_log_df.shape[0]]['template_id'].values)

In [24]:
token2count[207] / train_log_df.shape[0]

0.009023160966228426

In [21]:
token2count2 = Counter()
token2count2.update(log_df.iloc[train_log_df.shape[0]:]['template_id'].values)

In [25]:
token2count2[207] / test_log_df.shape[0]

0.026021290146483488

In [27]:
token2count

Counter({207: 4354,
         208: 3481,
         209: 111453,
         210: 631,
         211: 10,
         212: 2,
         213: 785,
         214: 83,
         215: 193,
         268: 3820,
         217: 273,
         218: 4,
         270: 967,
         220: 3,
         221: 98,
         222: 3988,
         223: 45,
         216: 5,
         224: 6,
         219: 4,
         225: 4,
         226: 29,
         317: 6624,
         227: 2666,
         228: 2549,
         229: 3574,
         230: 96,
         231: 49,
         232: 207,
         233: 3530,
         234: 21079,
         235: 88,
         236: 1164,
         237: 17387,
         238: 1412,
         239: 22,
         240: 1526,
         241: 7824,
         242: 1911,
         243: 50,
         244: 357,
         245: 833,
         246: 633,
         247: 19,
         248: 11,
         249: 158,
         250: 37,
         251: 4,
         252: 973,
         253: 874,
         254: 93,
         255: 136,
         256: 1,
    

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter

In [2]:
label1 = pd.read_csv('./data/preliminary_train_label_dataset.csv')
label2 = pd.read_csv('./data/preliminary_train_label_dataset_s.csv')
label_df = pd.concat([label1, label2]).reset_index(drop=True)

In [3]:
train_log_df = pd.read_csv('./data/preliminary_sel_log_dataset.csv')
test_log_df = pd.read_csv('./data/preliminary_sel_log_dataset_a.csv')

log_df = pd.concat([train_log_df, test_log_df]).reset_index(drop=True)

In [4]:
submit_df = pd.read_csv('./data/preliminary_submit_dataset_a.csv')

In [5]:
log_df['time'] = pd.to_datetime(log_df['time'])

In [8]:
label_df['fault_time'] = pd.to_datetime(label_df['fault_time'])

In [13]:
label_df.head()

,sn,fault_time,label
0,SERVER_25698,2020-10-09 13:43:00,0
1,SERVER_25699,2020-08-25 18:50:00,0
2,SERVER_25712,2020-03-16 13:20:00,0
3,SERVER_25708,2020-07-25 12:44:00,0
4,SERVER_25711,2020-03-16 16:51:00,0


In [31]:
for i in tqdm(range(label_df.shape[0])):
    row = label_df.iloc[i]
    sn = row['sn']
    start_fault_time = row['start_fault_time']
    fault_time = row['fault_time']
    tmp = log_df[log_df['sn'] == sn]
    tmp = tmp[(tmp['time'] >= start_fault_time) & (tmp['time'] <= fault_time)].sort_values(by='time')


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16669/16669 [08:28<00:00, 32.79it/s]


In [32]:
from drain3 import TemplateMiner #开源在线日志解析框架
from drain3.file_persistence import FilePersistence
from drain3.template_miner_config import TemplateMinerConfig

In [33]:
config = TemplateMinerConfig()
config.load('./drain3.ini') ## 这个文件在drain3的github仓库里有
config.profiling_enabled = False

In [34]:
drain_file = 'comp_a_sellog'
persistence = FilePersistence(drain_file + '.bin')
template_miner = TemplateMiner(persistence, config=config)

In [35]:
for msg in log_df.msg.tolist():
    template_miner.add_log_message(msg)
temp_count = len(template_miner.drain.clusters)

In [36]:
template_dic = {}
size_list = []
for cluster in template_miner.drain.clusters:
    size_list.append(cluster.size)

In [41]:
for cluster in template_miner.drain.clusters: ## 把符合要求的模板存下来
    template_dic[cluster.cluster_id] = cluster.size

In [43]:
temp_count_f = len(template_dic)

In [44]:
def match_template(df, template_miner, template_dic):
    msg = df.msg
    cluster = template_miner.match(msg) # 匹配模板，由开源工具提供
    if cluster and cluster.cluster_id in template_dic:
        df['template_id'] = cluster.cluster_id # 模板id
        df['template'] = cluster.get_template() # 具体模板
    else:
        df['template_id'] = 'None' # 没有匹配到模板的数据也会记录下来，之后也会用作一种特征。
        df['template'] = 'None'
    return df

In [45]:
log_df = log_df.apply(match_template, template_miner=template_miner, template_dic=template_dic, axis=1)

In [49]:
log_df.to_csv('./log_template.csv', index=False)

In [48]:
label_df['start_fault_time'] = label_df['fault_time'] - pd.Timedelta(pd.offsets.Hour(3))

for i in range(label_df.shape[0]):
    row = label_df.iloc[i]
    sn = row['sn']
    start_fault_time = row['start_fault_time']
    fault_time = row['fault_time']
    tmp = log_df[log_df['sn'] == sn]
    tmp = tmp[(tmp['time'] >= start_fault_time) & (tmp['time'] <= fault_time)].sort_values(by='time')
    print(tmp)
    break

               sn                time  \
129  SERVER_25698 2020-10-09 10:45:41   
75   SERVER_25698 2020-10-09 10:48:56   
5    SERVER_25698 2020-10-09 10:52:10   
124  SERVER_25698 2020-10-09 10:55:24   
223  SERVER_25698 2020-10-09 10:58:44   
158  SERVER_25698 2020-10-09 11:01:57   
236  SERVER_25698 2020-10-09 11:05:14   
246  SERVER_25698 2020-10-09 11:08:36   
7    SERVER_25698 2020-10-09 11:11:53   
169  SERVER_25698 2020-10-09 11:15:07   
177  SERVER_25698 2020-10-09 11:18:26   
201  SERVER_25698 2020-10-09 11:21:43   
153  SERVER_25698 2020-10-09 11:24:59   
64   SERVER_25698 2020-10-09 11:28:13   
40   SERVER_25698 2020-10-09 11:31:25   
258  SERVER_25698 2020-10-09 11:34:34   
99   SERVER_25698 2020-10-09 11:37:45   
175  SERVER_25698 2020-10-09 11:41:01   
13   SERVER_25698 2020-10-09 11:44:10   
254  SERVER_25698 2020-10-09 11:47:21   
15   SERVER_25698 2020-10-09 11:50:38   
145  SERVER_25698 2020-10-09 11:53:53   
182  SERVER_25698 2020-10-09 11:57:06   
66   SERVER_2569

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter

In [2]:
train_log_df = pd.read_csv('./data/preliminary_sel_log_dataset.csv')
test_log_df = pd.read_csv('./data/preliminary_sel_log_dataset_a.csv')
add_log_df = pd.read_csv('./data/additional_sel_log_dataset.csv')

In [3]:
label1 = pd.read_csv('./data/preliminary_train_label_dataset.csv')
label2 = pd.read_csv('./data/preliminary_train_label_dataset_s.csv')

label_df = pd.concat([label1, label2]).reset_index(drop=True)

In [4]:
log_df = pd.concat([train_log_df, test_log_df, add_log_df]).reset_index(drop=True)

In [5]:
data = []
for i in tqdm(range(log_df.shape[0])):
    msg = [item.strip() for item in log_df.iloc[i]['msg'].lower().strip().split(' ')]
    new_msg = []
    for item in msg:
        new_msg += item.split('_')
    msg = new_msg
    new_msg = []
    for item in msg:
        new_msg += item.split(':')
    msg = new_msg
    new_msg = []
    for item in msg:
        new_msg += item.split('/')
    msg = new_msg
    for kw in ['hdd', 'disk', 'cpu', 'fan', 'dimm']:
        new_msg = []
        for item in msg:
            if item.startswith(kw):
                new_msg += [kw, item[len(kw):]]
            else:
                new_msg += [item]
        msg = new_msg
    msg = list(filter(lambda x: len(x) > 0, msg))
    data.append(msg)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 1226201/1226201 [01:37<00:00, 12529.42it/s]


In [19]:
maxlen = 0
msg = ''
for item in data:
    if len(item) > maxlen and len(item) < 30:
        msg = item
        maxlen = len(item)
maxlen

25

In [21]:
log_df.groupby('sn').size()

sn
000d33b21436     2
005c5a9218ba    10
0079283bde6e     1
007bdf23b62f    19
00a577a8e54f     6
                ..
ffbf46b4af21     3
ffc229b6cd9a     2
ffd44698a52b     2
fff73a9e5bd5     6
fffd22fffe19    17
Length: 28237, dtype: int64

In [7]:
cnt = 0
for sn, group in log_df.groupby('sn'):
    group = group.sort_values(by='time')
    sub_label_df = label_df[label_df['sn'] == sn].sort_values(by='fault_time')
    if group.shape[0] > 200:
        print(group[['time', 'msg']])
        print(sub_label_df)
        break

                       time                                                msg
322999  2020-06-16 02:01:59   Memory CPU1D0_DIMM_Stat | Correctable ECC | A...
323317  2020-06-16 02:03:25   Memory CPU1D0_DIMM_Stat | Correctable ECC | A...
323378  2020-06-16 02:05:39   Memory CPU1D0_DIMM_Stat | Correctable ECC | A...
324291  2020-06-16 02:06:10   Memory CPU1D0_DIMM_Stat | Correctable ECC | A...
325751  2020-06-16 02:06:56   Memory CPU1D0_DIMM_Stat | Correctable ECC | A...
...                     ...                                                ...
325960  2020-06-16 15:21:57   Processor CPU1_Status | Configuration Error |...
325220  2020-06-16 15:21:57   Memory CPU1D0_DIMM_Stat | Uncorrectable ECC |...
323370  2020-06-16 15:50:06   Memory CPU1D0_DIMM_Stat | Correctable ECC | A...
323897  2020-06-16 16:39:44   Memory CPU1D0_DIMM_Stat | Correctable ECC | A...
322526  2020-06-16 16:48:04   System ACPI Power State ACPI_PWR_Status | S4/...

[1078 rows x 2 columns]
                sn         

In [22]:
train_log_df

,sn,time,msg,server_model
0,SERVER_25698,2020-10-09 08:32:21,System Boot Initiated BIOS_Boot_Up | State As...,SM0
1,SERVER_25698,2020-10-09 07:43:48,System Boot Initiated BIOS_Boot_Up | State As...,SM0
2,SERVER_25698,2020-10-09 08:16:22,System Boot Initiated BIOS_Boot_Up | State As...,SM0
3,SERVER_25698,2020-10-09 05:46:41,System Boot Initiated BIOS_Boot_Up | State As...,SM0
4,SERVER_25698,2020-10-09 12:59:13,System Boot Initiated BIOS_Boot_Up | State As...,SM0
...,...,...,...,...
482531,SERVER_24971,2020-10-04 14:48:11,Drive Slot / Bay BP2_HDD13_Status | Predictiv...,SM102
482532,SERVER_24971,2020-11-12 02:15:25,Drive Slot / Bay BP2_HDD25_Status | Predictiv...,SM102
482533,SERVER_24971,2020-11-12 17:10:34,Drive Slot / Bay BP2_HDD25_Status | Predictiv...,SM102
482534,SERVER_24971,2020-11-12 18:17:57,Drive Slot / Bay BP1_HDD27_Status | Predictiv...,SM102


In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
log_df = pd.read_csv('./data/preliminary_sel_log_dataset.csv')
test_log_df = pd.read_csv('./data/preliminary_sel_log_dataset_a.csv')

In [10]:
log_df = pd.concat([log_df, test_log_df]).reset_index(drop=True)

In [11]:
for sn, group in log_df.groupby('sn'):
    if np.unique(group['server_model'].values).shape[0] > 1:
        print(sn, np.unique(group['server_model'].values))

In [12]:
np.unique(log_df['server_model'])

array(['SM0', 'SM100', 'SM102', 'SM103', 'SM106', 'SM107', 'SM109',
       'SM11', 'SM12', 'SM13', 'SM14', 'SM15', 'SM16', 'SM17', 'SM18',
       'SM19', 'SM20', 'SM21', 'SM22', 'SM23', 'SM24', 'SM25', 'SM26',
       'SM27', 'SM28', 'SM29', 'SM3', 'SM30', 'SM31', 'SM32', 'SM33',
       'SM34', 'SM35', 'SM36', 'SM37', 'SM38', 'SM39', 'SM4', 'SM40',
       'SM41', 'SM45', 'SM46', 'SM49', 'SM5', 'SM50', 'SM51', 'SM52',
       'SM53', 'SM54', 'SM55', 'SM56', 'SM57', 'SM58', 'SM59', 'SM6',
       'SM60', 'SM61', 'SM63', 'SM65', 'SM66', 'SM67', 'SM68', 'SM69',
       'SM70', 'SM71', 'SM73', 'SM74', 'SM75', 'SM76', 'SM77', 'SM78',
       'SM79', 'SM8', 'SM82', 'SM83', 'SM85', 'SM87', 'SM88', 'SM9',
       'SM91', 'SM92', 'SM93', 'SM94', 'SM95', 'SM96', 'SM98', 'SM99'],
      dtype=object)

In [13]:
label1 = pd.read_csv('./data/preliminary_train_label_dataset.csv')
label2 = pd.read_csv('./data/preliminary_train_label_dataset_s.csv')

In [14]:
label_df = pd.concat([label1, label2]).reset_index(drop=True)

In [15]:
for i in range(4):
    print(label_df[label_df['label'] == i].shape[0])

1476
3387
9343
2463


In [16]:
pred_df = pd.read_csv('./preliminary_pred_a.csv')

In [17]:
for i in range(4):
    print(pred_df[pred_df['label'] == i].shape[0])

74
710
1748
479
